In [1]:

import pandas as pd
import re

In [2]:

bank_statement_df = pd.read_excel(r"D:\NPS_phase1\nps-reconciliation\output\BANK STMT PHASE 4\Bank_Agriculture.xlsx", skiprows=8)


In [3]:
bank_statement_df.head()

,S.NO,Nepali Value Date,Value Date,Nepali Booking Date,Booking Date,Trans Ref,Narrative,DEBIT,CREDIT,Balance,Total Turnover
0,NaN,NaN,NaN,NaN,NaN,NaN,Balance at Period Start,NaN,NaN,"'-15,696,883.94'",NaN
1,1.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246LK86L,100004007250,0,3720.0,"'-15,693,163.94'",NaN
2,2.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246JJ1KK,NPS-IF-3173579,"'-3,200.00'",0.0,"'-15,696,363.94'",NaN
3,3.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246GM2PX,NPS-IF-3173632,"'-16,397.00'",0.0,"'-15,712,760.94'",NaN
4,4.0,20800517.0,20230903.0,20800517.0,20230903.0,FT232466JKSK,100004007311,0,500.0,"'-15,712,260.94'",NaN


In [4]:
bank_statement_df.drop(bank_statement_df.index[0],inplace=True)
bank_statement_df.drop(bank_statement_df.index[-1], inplace=True)
bank_statement_df["DEBIT"] = bank_statement_df["DEBIT"].str.replace("'","").str.replace(",","")
bank_statement_df["DEBIT"].fillna("0.0", inplace=True)
bank_statement_df




,S.NO,Nepali Value Date,Value Date,Nepali Booking Date,Booking Date,Trans Ref,Narrative,DEBIT,CREDIT,Balance,Total Turnover
1,1.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246LK86L,100004007250,0.0,3720.0,"'-15,693,163.94'",NaN
2,2.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246JJ1KK,NPS-IF-3173579,-3200.00,0.0,"'-15,696,363.94'",NaN
3,3.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246GM2PX,NPS-IF-3173632,-16397.00,0.0,"'-15,712,760.94'",NaN
4,4.0,20800517.0,20230903.0,20800517.0,20230903.0,FT232466JKSK,100004007311,0.0,500.0,"'-15,712,260.94'",NaN
5,5.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246G6HR0,100004007331,0.0,300.0,"'-15,711,960.94'",NaN
...,...,...,...,...,...,...,...,...,...,...,...
428,428.0,20800518.0,20230904.0,20800518.0,20230904.0,FT2324707W4L,NPS-IF-3186339,-1300.00,0.0,"'-18,922,204.40'",NaN
429,429.0,20800518.0,20230904.0,20800518.0,20230904.0,FT232474CPHV,NPS-IF-3186360,-18000.00,0.0,"'-18,940,204.40'",NaN
430,430.0,20800518.0,20230904.0,20800518.0,20230904.0,FT23247H4FWS,NPS-IF-3186379,-3000.00,0.0,"'-18,943,204.40'",NaN
431,431.0,20800518.0,20230904.0,20800518.0,20230904.0,FT23247YMM5P,NPS-IF-3186436,-190.00,0.0,"'-18,943,394.40'",NaN


In [14]:
bank_statement_df['DEBIT']= bank_statement_df['DEBIT'].astype(float)

In [15]:
for index, row in bank_statement_df.iterrows():
    if "NPS-IF-" in str(row["Narrative"]):
        id = re.findall(r'[0-9]{7}',str(row["Narrative"]))
        bank_statement_df.loc[index, 'Transaction ID'] = id[0]

    elif "10000" in str(row["Narrative"]):
        id = re.findall(r'10*[0-9]{7}',str(row["Narrative"]))
        bank_statement_df.loc[index, 'Transaction ID'] = id[0].replace("10000","")
    
    elif "FTMS-" in str(row["Narrative"]):
        id = re.findall(r'[0-9]{6}',str(row["Narrative"]))
        bank_statement_df.loc[index, 'Transaction ID'] = id[0]
   
        
        # elif re.findall(r'[0-9]{7}', str(row["Narrative"]))[0] in str(row["Narrative"]):
        #     id = re.findall(r'[0-9]{7}', str(row["Narrative"]))
        #     bank_statement_df.loc[index, 'Transaction ID'] = id[0]
            
        
            
            
            

In [16]:
def standard_format(row):
        """
        Standardize transaction amount and mode.

        Classify transactions as debit (DR) or credit (CR).
        """
        if abs(row['DEBIT']) > 0:
            mode = 'DR'
            amount = row['DEBIT']
        elif row['CREDIT'] > 0:
            mode = 'CR'
            amount = row['CREDIT']
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])

In [17]:
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format, axis=1)   

In [18]:
bank_statement_df.head()

,S.NO,Nepali Value Date,Value Date,Nepali Booking Date,Booking Date,Trans Ref,Narrative,DEBIT,CREDIT,Balance,Total Turnover,Transaction ID,Mode,Amount
1,1.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246LK86L,100004007250,0.0,3720.0,"'-15,693,163.94'",NaN,4007250,CR,3720.0
2,2.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246JJ1KK,NPS-IF-3173579,-3200.0,0.0,"'-15,696,363.94'",NaN,3173579,DR,-3200.0
3,3.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246GM2PX,NPS-IF-3173632,-16397.0,0.0,"'-15,712,760.94'",NaN,3173632,DR,-16397.0
4,4.0,20800517.0,20230903.0,20800517.0,20230903.0,FT232466JKSK,100004007311,0.0,500.0,"'-15,712,260.94'",NaN,4007311,CR,500.0
5,5.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246G6HR0,100004007331,0.0,300.0,"'-15,711,960.94'",NaN,4007331,CR,300.0


In [7]:
bank_statement_df.columns

Index(['S.NO', 'Nepali Value Date', 'Value Date', 'Nepali Booking Date',
       'Booking Date', 'Trans Ref', 'Narrative', 'DEBIT', 'CREDIT', 'Balance',
       'Total Turnover', 'Transaction ID'],
      dtype='object')

In [6]:
bank_statement_df

,S.NO,Nepali Value Date,Value Date,Nepali Booking Date,Booking Date,Trans Ref,Narrative,DEBIT,CREDIT,Balance,Total Turnover,Transaction ID
1,1.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246LK86L,100004007250,0.0,3720.0,"'-15,693,163.94'",NaN,4007250
2,2.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246JJ1KK,NPS-IF-3173579,-3200.00,0.0,"'-15,696,363.94'",NaN,3173579
3,3.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246GM2PX,NPS-IF-3173632,-16397.00,0.0,"'-15,712,760.94'",NaN,3173632
4,4.0,20800517.0,20230903.0,20800517.0,20230903.0,FT232466JKSK,100004007311,0.0,500.0,"'-15,712,260.94'",NaN,4007311
5,5.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246G6HR0,100004007331,0.0,300.0,"'-15,711,960.94'",NaN,4007331
...,...,...,...,...,...,...,...,...,...,...,...,...
428,428.0,20800518.0,20230904.0,20800518.0,20230904.0,FT2324707W4L,NPS-IF-3186339,-1300.00,0.0,"'-18,922,204.40'",NaN,3186339
429,429.0,20800518.0,20230904.0,20800518.0,20230904.0,FT232474CPHV,NPS-IF-3186360,-18000.00,0.0,"'-18,940,204.40'",NaN,3186360
430,430.0,20800518.0,20230904.0,20800518.0,20230904.0,FT23247H4FWS,NPS-IF-3186379,-3000.00,0.0,"'-18,943,204.40'",NaN,3186379
431,431.0,20800518.0,20230904.0,20800518.0,20230904.0,FT23247YMM5P,NPS-IF-3186436,-190.00,0.0,"'-18,943,394.40'",NaN,3186436


In [9]:
bank_statement_df['Transaction ID'].count()

429

In [10]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Value Date'], format="%Y%m%d").dt.date

In [11]:
bank_statement_df.head()

,S.NO,Nepali Value Date,Value Date,Nepali Booking Date,Booking Date,Trans Ref,Narrative,DEBIT,CREDIT,Balance,Total Turnover,Transaction ID,Date
0,NaN,NaN,NaN,NaN,NaN,NaN,Balance at Period Start,0.0,NaN,"'-15,696,883.94'",NaN,NaN,NaT
2,2.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246JJ1KK,NPS-IF-3173579,-3200.00,0.0,"'-15,696,363.94'",NaN,3173579,2023-09-03
3,3.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246GM2PX,NPS-IF-3173632,-16397.00,0.0,"'-15,712,760.94'",NaN,3173632,2023-09-03
4,4.0,20800517.0,20230903.0,20800517.0,20230903.0,FT232466JKSK,100004007311,0.0,500.0,"'-15,712,260.94'",NaN,4007311,2023-09-03
5,5.0,20800517.0,20230903.0,20800517.0,20230903.0,FT23246G6HR0,100004007331,0.0,300.0,"'-15,711,960.94'",NaN,4007331,2023-09-03


In [6]:
bank_statement_df['Transaction ID'].values

array(['2842471', '3808187', '2842598', '2842655', '2842668', '2842687',
       '2842746', '2842768', '2842787', '2842816', '2842840', '2842850',
       '2842857', '2842892', '2842895', '2843049', '2843061', '2843079',
       '2843087', '2843178', '3808502', '3808545', '2843290', '2843301',
       '156685', '156768', '3808604', '156795', '156804', '3808622',
       '156907', '156919', '156932', '2843396', '3808790', '3808868',
       '2843485', '2843496', '2843531', '3808973', '3808983', '2843616',
       '2843628', '2843631', '2843648', '2843689', '2843730', '2843756',
       '2843780', '2843791', '3809194', '2843836', '2843840', '157063',
       '2843865', '2843876', '3809264', '3809333', '2844036', '2844037',
       '157093', '3809395', '2844066', '3809426', '2844146', '2844257',
       '2844268', '3809561', '3809590', '2844384', '2844386', '2844407',
       '3809597', '3809606', '2844441', '2844461', '2844462', '2844463',
       '2844518', '3809690', '2844569', '2844691', '2844739'